In [1]:
import pandas as pd
import numpy as np
import re
from nltk import sent_tokenize, word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from gensim.models import Word2Vec

In [2]:
df = pd.read_csv('data.csv', encoding = 'ISO 8859-1')

In [3]:
df.head()

,Unnamed: 0,topic,text
0,0,Student,In het kader van kernfusie op aarde: MAAK JE ...
1,1,InvestmentBanking,"Just so you know, this blog isn't about being ..."
2,2,Education,"I I donÃÂ¢ÃÂÃÂt remember his name, but I..."
3,3,Science,so there they were in the cotton candy shack. ...
4,4,Student,By: TIRI!!! I was sitting in the police sta...


In [4]:
df = df[['topic', 'text']]

In [5]:
df.head()

,topic,text
0,Student,In het kader van kernfusie op aarde: MAAK JE ...
1,InvestmentBanking,"Just so you know, this blog isn't about being ..."
2,Education,"I I donÃÂ¢ÃÂÃÂt remember his name, but I..."
3,Science,so there they were in the cotton candy shack. ...
4,Student,By: TIRI!!! I was sitting in the police sta...


In [6]:
df.text[50][:500]

'Ã\x83Â¢Ã\x82Â\x80Ã\x82Â\x9cIÃ\x83Â¢Ã\x82Â\x80Ã\x82Â\x99m sorry, what?Ã\x83Â¢Ã\x82Â\x80Ã\x82Â\x9d I mumbled as I looked up, meeting eyes with my waitress. IÃ\x83Â¢Ã\x82Â\x80Ã\x82Â\x99m not entirely sure how long she had been standing there or how many times she had asked her question. Ã\x83Â¢Ã\x82Â\x80Ã\x82Â\x9cOh um, right. Could I get a mocha?Ã\x83Â¢Ã\x82Â\x80Ã\x82Â\x9d  Ã\x83Â¢Ã\x82Â\x80Ã\x82Â\x9cSure thing, Hon. No problem,Ã\x83Â¢Ã\x82Â\x80Ã\x82Â\x9d she replied in a motherly voice as she turned to head back to the counter.  Ã\x83Â¢Ã\x82Â\x80Ã\x82Â\x9cOh, and a little extra chocolate, please!Ã\x83Â¢Ã\x82Â\x80Ã\x82Â\x9d I called after her. '

In [7]:
df['text'] = df['text'].replace('[^a-zA-Z0-9,.!? \'\"]', '', regex = True)
df['text'] = df['text'].replace('  ', ' ', regex = True)

In [8]:
df.text[50][:500]

'Im sorry, what? I mumbled as I looked up, meeting eyes with my waitress. Im not entirely sure how long she had been standing there or how many times she had asked her question. Oh um, right. Could I get a mocha? Sure thing, Hon. No problem, she replied in a motherly voice as she turned to head back to the counter. Oh, and a little extra chocolate, please! I called after her. It wasnt that I was a chocolate freak. I just had to cover up the bite of the espresso. Too much of that, and my stomach w'

In [9]:
stopwords_english = stopwords.words('english')
lemmatizer = WordNetLemmatizer()

In [10]:
def get_title(text):
    try:
        details = [
            [
                lemmatizer.lemmatize(word)
                for word in word_tokenize(re.sub('[^a-zA-Z ]', '', sentence))
                if word not in stopwords_english
            ]
            for sentence in sent_tokenize(text)
        ]

        word2vec = Word2Vec(details, min_count = 1)

        word2vec_counts = dict()
        for word in word2vec.wv.key_to_index:
            if word.lower() not in stopwords_english:
                word2vec_counts[word] = word2vec.wv.get_vecattr(word, 'count')

        word2vec_counts = sorted(word2vec_counts.items(), key = lambda item: item[1], reverse = True)[0:10]

        return ' '.join(dict(word2vec_counts).keys())
    except:
        return np.NaN

In [11]:
df.head()

,topic,text
0,Student,In het kader van kernfusie op aarde MAAK JE EI...
1,InvestmentBanking,"Just so you know, this blog isn't about being ..."
2,Education,"I I dont remember his name, but I remember the..."
3,Science,so there they were in the cotton candy shack. ...
4,Student,By TIRI!!! I was sitting in the police statio...


In [12]:
df['title'] = df.text.apply(get_title)

In [13]:
df.head()

,topic,text,title
0,Student,In het kader van kernfusie op aarde MAAK JE EI...,uranium U bomb Hbomb nuclear plant like get on...
1,InvestmentBanking,"Just so you know, this blog isn't about being ...",urlLink war think Korea one Im US could countr...
2,Education,"I I dont remember his name, but I remember the...",Mathew Autism would first classroom teacher ki...
3,Science,so there they were in the cotton candy shack. ...,john bo jangle got really everyone like cuz co...
4,Student,By TIRI!!! I was sitting in the police statio...,didnt went told day could saw started made guy...


In [14]:
df.title

0       uranium U bomb Hbomb nuclear plant like get on...
1       urlLink war think Korea one Im US could countr...
2       Mathew Autism would first classroom teacher ki...
3       john bo jangle got really everyone like cuz co...
4       didnt went told day could saw started made guy...
                              ...                        
2604    yes know people dont Yes sex think love one fr...
2605    Scott Donnie fire dont know right think one th...
2606    people colony Assignment one Americans America...
2607    America right vote Mr President got promise sa...
2608    urlLink July dont Ashley Jennifer PM thing thi...
Name: title, Length: 2609, dtype: object

In [15]:
df.save_to_csv('data_processsed.csv')

AttributeError: 'DataFrame' object has no attribute 'save_to_csv'